In [14]:
import pickle as pk
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits
# from mpl_toolkits import mplot3d
from matplotlib import cm
import matplotlib.patches as patches
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import numpy as np
import scipy
import math
import pandas as pd
import os
import itertools
import glob

import torch
torch.set_default_dtype(torch.float64)

%matplotlib inline

class ScalarFormatterForceFormatZero(ticker.ScalarFormatter):
    def _set_format(self):  # Override function that finds format to use
        self.format = '%1.0f'  # Give format here

class ScalarFormatterForceFormatOne(ticker.ScalarFormatter):
    def _set_format(self):
        self.format = '%1.1f'

class ScalarFormatterForceFormat(ticker.ScalarFormatter):
    def _set_format(self):
        self.format = '%1.2f'
        
class ScalarFormatterForceFormatThree(ticker.ScalarFormatter):
    def _set_format(self):
        self.format = '%1.3f'

In [17]:
%load_ext autoreload
%autoreload all --print

import sys
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy'))
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy/tnpy'))

import tnpy as tp
from tnpy import GTensor
from tnpy.mps import FermiMPS as fMPS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# our fMPS tensor convention:
# 
#  0 --<--*--<-- 1
#         ｜2
#         v

gt_dual = (0, 1, 0)
gt_shape = ((3, 3), (3, 3), (2, 2))

fmps = fMPS.rand(n=16, dual=gt_dual, shape=gt_shape)
fmps_dagger = fmps.dagger()
print(fmps.inner_product(fmps_dagger, fmps), fmps.inner_product(fmps, fmps_dagger))

fmps_0 = fMPS.rand(n=16, dual=gt_dual, shape=gt_shape)
print(fmps.inner_product(fmps_0.dagger(), fmps))

lc_mps = fmps.left_canonical()
rc_mps = fmps.right_canonical()

print('left inner product:', fmps.inner_product(lc_mps.dagger(), lc_mps))
print('right inner product:', fmps.inner_product(rc_mps.dagger(), rc_mps))

print('Fidelity:', fmps.fidelity(lc_mps, rc_mps))

tensor(6.48920e+20-9175.77196j) tensor(6.48920e+20+886.41808j)
tensor(3.02967e+20-6.17871e+19j)
left inner product: tensor(6.00000+1.28005e-16j)
right inner product: tensor(6.00000-2.66738e-16j)
Fidelity: tensor(0.00250-5.32525e-20j)


In [20]:
num = 16
gt_dual = (0, 1, 0)
ref_fmps = fMPS.rand_obc(n=num, dual=gt_dual, max_shape=(4, 4)).left_canonical()
fmps = fMPS.rand_obc(n=num, dual=gt_dual, max_shape=(4, 4)).left_canonical()
# fmps.sweep_lr(ref_fmps)

In [ ]:
# finite open MPS

